# Certify with LLMs

- **Goal**: After `hybrid_retrieval.ipynb`, take the retrieved results and classify as either prediction or non-predicion.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from prediction_properties import PredictionProperties
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# Load Data

In [3]:
new_file = True
base_data_path = os.path.join(notebook_dir, '../data/')
hybrid_retrieval_results_path = os.path.join(base_data_path, 'hybrid_retrieval/')

if new_file == True: 
    new_results_file_name = "retrieved_results-v1.csv"
    path = os.path.join(hybrid_retrieval_results_path, new_results_file_name)
    save_file_type = "csv"
    df = DataProcessing.load_from_file(path, save_file_type)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
# else: 
#     path = os.path.join(notebook_dir, "../data/rag/retrieved/", "text_label_name_meta_data-v1.csv")
#     # prefix = "text_label_metadata"
#     save_file_type = "csv"
#     df = DataProcessing.load_from_file(path, save_file_type)
#     df.drop(['Unnamed: 0'], axis=1, inplace=True)
#     df

df

,id,content,blob,meta,score,embedding,sparse_embedding,query
0,c8250eb1bfc021b035d71ba172e262a21ab427b756d6a7776cbbdf3bdc56e96c,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",NaN,"{'sentiment': 'neutral', 'source_id': '63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.048373,"[0.04417748004198074, -0.03718455508351326, -0.026471151039004326, -0.014962117187678814, 0.046283308416604996, -0.02831903286278248, -0.07162532210350037, 0.05365421622991562, -0.04093141108751297, 0.0298501905053854, 0.009826543740928173, 0.01519095804542303, 0.034347400069236755, -0.028681518509984016, -0.03954571112990379, -0.002502115909010172, -0.006628081202507019, -0.12832355499267578, -0.09077678620815277, -0.07121571153402328, 0.048768915235996246, -0.08011864125728607, 0.010965651832520962, 0.004186794627457857, 0.08360739052295685, -0.05175008624792099, -0.015410348773002625, 0.06874939799308777, -0.05621153116226196, -0.10414990782737732, -0.04159145802259445, -0.011647788807749748, -0.017109237611293793, 0.047060608863830566, 0.06767600029706955, 0.056333694607019424, 0.0...",NaN,"You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n ..."
1,9e8da179f1923e36a0092be7b7b424377e1daae0870033d8c91988bd15b3c277,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,NaN,"{'sentiment': 'positive', 'source_id': '39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.041144,"[-0.035207055509090424, -0.04257950559258461, 0.034338660538196564, 0.04421427473425865, 0.07892248779535294, 0.0337398499250412, 0.013001815415918827, 0.0477336049079895, -0.06594444811344147, 0.015794288367033005, 0.020301755517721176, -0.025846945121884346, 0.04249538853764534, 0.0024806081783026457, 0.0225980244576931, -0.003781020874157548, 0.025006961077451706, -0.06871230900287628, -0.11034145206212997, -0.042451225221157074, -0.007218378130346537, -0.005378246307373047, -0.019224535673856735, -0.04748206585645676, 0.016703318804502487, -0.08213851600885391, -0.06476438790559769, 0.050822217017412186, -0.014659274369478226, -0.09981179237365723, -0.04532046988606453, 0.038232963532209396, 0.09121283888816833, -0.0176495760679245, -0.027979986742138863, 0.024657418951392174, -0.0...",NaN,"You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n ..."
2,084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy 

## Filter for No Labels

In [4]:
if new_file == False:
    na_filt = df['llm_label'].isna()
    no_labels_df = df[na_filt]
    no_labels_df

In [5]:
# no_labels_df['meta']

## LLM Certification

In [6]:
tgmf = TextGenerationModelFactory()


# Groq Cloud (https://console.groq.com/docs/overview)
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

# models  = [llama_318b_instant_generation_model]
models  = [llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]


# NaviGator
# llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct') 
# mistral_small_31 = tgmf.create_instance('mistral-small-3.1') 
# models = [llama_31_70b_instruct, mistral_small_31]

In [7]:
prediction_properties = PredictionProperties.get_prediction_properties()
prediction_requirements = PredictionProperties.get_requirements()
system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
    1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
    2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
    3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
"""
non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
    1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
    2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
    3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
"""
# goal_prompt = "Given the above, identify the prediction."

base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
    {prediction_properties}
    Enforce: {prediction_requirements}
    Know: {prediction_examples_prompt}
    Know: {non_prediction_examples_prompt}

"""
base_prompt

'You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: \n\n     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n        1. <p_s>\n            - Defined as: \n                - Source entity that states the <p>\n            - Characteristics:\n                - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                - An associated organization\n                - Named entity: Person, organization\n                - Part of speech: Noun\n            - Examples:\n                1. A person with a name only: Detravious\n                2. A person with a profile name: FitToJesus\n                3. A person with a gender only: He\n                4. A person with a domain specific title: reporter, analyst, expert, top executive, senior level person, etc \n                5. A pe

In [16]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt} and the sentence '{data}', classify if the {data} is a prediction or non-prediction. Do NOT generate anything more than these to classes (prediction or non-prediction). Do not state a reason for your classification of the text. Do not regenerate the text either
        """
        idx = 1
        if idx == 1:
              print(f"Prompt: {prompt}")
              idx = idx + 1

        input_prompt = model.user(prompt)
        # print(input_prompt)  

        raw_text_llm_generation = model.chat_completion([input_prompt])
        # print(raw_text_llm_generation)

        for line in raw_text_llm_generation.split("\n"):
            return line

In [17]:
# content : meta :: text : meta_data
results = []
for idx, row in df.iterrows():
    text = row['content']
    # meta_data = row.drop(labels=['text', 'llm_label', 'llm_name'])
    meta_data = row['meta']
    print(idx, text)

    if idx == 1:
        for model in models:
            # print(idx, result)
            llm_label = llm_certifier(text, base_prompt, model)
            print('\t', model.__name__(), '---', llm_label)
            result = (text, llm_label, model.__name__(), meta_data)
            # result = (text, llm_label, model.__name__())
            results.append(result)

0 According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
1 With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
Prompt:  Given this: You are an expert at identifying specific types of sentences by knowing the sentence format. The sentence format is based on: 

     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

        1. <p_s>
            - Defined as: 
                - Source entity that states the <p>
            - Characteristics:
                - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.
                - An associated organization
                - Named entity: Person, organization
                - Part of speech: Noun

In [10]:
# results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'llm_label', 'llm_name', 'meta_data'])
# results_with_llm_label_df
new_results_df = pd.DataFrame(results, columns=['text', 'llm_label', 'llm_name', 'meta_data'])
new_results_df

,text,llm_label,llm_name,meta_data
0,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.1-8b-instant,"{'sentiment': 'neutral', 'source_id': '63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
1,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Non-prediction,llama-3.3-70b-versatile,"{'sentiment': 'neutral', 'source_id': '63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
2,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",safe,meta-llama/llama-guard-4-12b,"{'sentiment': 'neutral', 'source_id': '63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
3,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,prediction,llama-3.1-8b-instant,"{'sentiment': 'positive', 'source_id': '39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
4,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,Prediction,llama-3.3-70b-versatile,"{'sentiment': 'positive', 'source_id': '39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
5,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,safe,meta-llama/llama-guard-4-12b,"{'sentiment': 'positive', 'source_id': '39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
6,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"{'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
7,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"{'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
8,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,safe,meta-llama/llama-guard-4-12b,"{'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}"
9,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Prediction.,llama-3.1-8b-instant,"{'sentiment': 'negative', 'source_id': 'a733e18a009c0cdef1ee85366b3506eb3f869588c69b0c4d604021aa6d34a1df', 'page_number': 1, 'split_id': 0, 'split_idx

In [11]:
# llm_labelled_df = df.dropna()
# llm_labelled_df

In [12]:
# updated_df = pd.concat([llm_labelled_df, new_results_df])
# updated_df

In [13]:
# path = os.path.join(notebook_dir, "../data/rag/retrieved/")
# prefix = "text_label_name_meta_data"
# save_file_type = "csv"
# DataProcessing.save_to_file(updated_df, path, prefix, save_file_type)